### This notebook clean up beauty product data and prepare some basic statistic features.

In [1]:
from difflib import SequenceMatcher
from ast import literal_eval
from collections import Counter
from tqdm import tqdm
import pandas as pd
import numpy as np
import re
import operator
import itertools
import gc

### Prepare ingredient information

Read ingredient dataframe:

In [2]:
cols = ['name','category','rating']
ingredient_df = pd.read_csv('../web_scraper/ingredients.csv', usecols=cols, converters={"category": literal_eval})
ingredient_df['name'] = ingredient_df['name'].str.strip()
ingredient_df['rating_num'] = ingredient_df['rating'].map({'Poor':0, 'Average':1, 'Good':2, 'GOOD':2, 'Best':3})
print("number of ingredient:",ingredient_df.shape[0])
ingredient_df.head()

number of ingredient: 1750


,name,category,rating,rating_num
0,"1, 2-Hexanediol",[Preservatives],Good,2
1,10-Hydroxydecanoic Acid,[Emollients],Good,2
2,4-T-butylcyclohexanol,"[Emollients, Skin-Soothing]",Good,2
3,Acacia farnesiana extract,"[Plant Extracts, Fragrance: Synthetic and Frag...",Poor,0
4,acacia senegal gum,"[Texture Enhancer, Plant Extracts, Skin-Soothing]",Good,2


Create a class where we can check if an ingredient matches our existing ingredient dictionary. if there is a match,
find the ingredient's rating and category.
* Initialize the class with ingredient rating dictionary and category dictionary.
* Given a list of ingredient, find best matching ingredient that rating and category are avaliable. This is done by evaluating the similarity metric between the name of all existing ingredient and the name of given ingredient (use python function SequenceMatcher). If the similarity is below a thresh, then the given ingredient is labeled as 'unknown'.
* After building up the matching dictionary, we can find an ingredient's matching, rating and category by calling the lookup function.

In [3]:
class look_up_ingredient():
    
    def __init__(self, rating_dict, category_dict):
        self.rating_dict = rating_dict
        self.rating_dict['unknown'] = np.nan
        
        self.category_dict = category_dict
        self.category_dict['unknown'] = []
        
        self.rating = set([value for value in self.rating_dict.values()])
        self.category = set([value for values in self.category_dict.values() for value in values])
        
        self.match_dict = {}
    
    def find_matching_ingredient(self, my_ingredients, thresh=0.2):

        self.match_dict = {}
        for ingredient in tqdm(my_ingredients):
            match_matric = {key : SequenceMatcher(None, key, ingredient).ratio() for key in self.rating_dict.keys()}
            best_match, best_metric = max(match_matric.items(), key=operator.itemgetter(1))
            if best_metric > thresh:
                self.match_dict[ingredient] = best_match
            else:
                self.match_dict[ingredient] = 'unknown'
                
    def lookup(self, ingredient, option=''):
        
        key = self.match_dict.get(ingredient, 'unknown')
        rating = self.rating_dict.get(key, -1)
        category = self.category_dict.get(key, [])
        
        if option == 'ingredient':
            return key
        elif option == 'rating':
            return rating
        elif option == 'category':
            return category
        else:
            return key, rating, category

Create ingredient class, note for ingredient with alias we will duplicate the record. 

For example, for "PEG/PPG-18/18 dimethicone" we will create three dict items, with different keys but same value.

In [4]:
ingredient_rating_dict = {name: row['rating_num'] for (idx, row) in ingredient_df.iterrows() for name in row['name'].split('/')}
ingredient_category_dict = {name: row['category'] for (idx, row) in ingredient_df.iterrows() for name in row['name'].split('/')}
lookup = look_up_ingredient(ingredient_rating_dict, ingredient_category_dict)

### Clean product data 

* Drop products that are not "chemical" products, like makeup brushes, cleaning devices.
* Split 'size' column to a number and unit, to unit conversion as necessary
* Compute 'price/size'
* Basic cleaning on ingredients:
    * split inactive and active ingredient
    * convert ingredients to a list
    * find number of inactive and active ingredient
    * check if the ingredients are in alphabatical order -- most companies like to list ingredient in a descending order of their quantity in the product, some companies just list ingredients alphabatically.
* Look up ingredients in our ingredient dictionary.
    * get a set of all unique ingredients in the products dataframe
    * find the match of all these ingredients
    * for all product, we loop over its ingredient list and look up the matching ingredient, rating and ingredient category
    * count how many ingredients in a product have a certain rating (how many ingredient rated as Good/Average etc.)
    * count how many ingredients in a product belongs to a certain category (how many antioxidants/sunscreen etc.)
    * compute average ingredient rating, potentially we want to take product's rank into consideration.

In [29]:
class product_df_cleaning:
    
    def __init__(self, df):
        self.df = df.copy(deep=True)
        
    def drop_rows(self, drop_dict):
        for col, values in drop_dict.items():
            self.df = self.df.loc[~self.df[col].isin(values)]
    
    def clean_price(self):
        # convert all price to float
        # compute price/size
        if self.df['price'].dtype != 'float':
            self.df['price'] = self.df['price'].apply(lambda x: x.replace(',','')).astype('float')
        self.df['avg_price'] = self.df['price']/self.df['size_num']
        
    def clean_size(self):
        # split size number and unit
        # convert fl.oz. to ml
        self.df['size_num'] = self.df['size'].apply(lambda x: float(str(x).split()[0]))
        self.df['size_unit'] = self.df['size'].apply(lambda x: str(x)[len(str(x).split()[0])+1:])
        self.df['size_num'].loc[self.df['size_unit']=='fl. oz.'] *= 29.5735
        self.df['size_num'] = self.df['size_num'].round()
        self.df['size_unit'].loc[self.df['size_unit']=='fl. oz.'] = 'ml'
        self.df['size_unit'].loc[self.df['size_unit']=='grams'] = 'gram'
        self.df['size_unit'].loc[~self.df['size_unit'].isin(['ml','gram',''])] = 'piece/other'
        
    def clean_ingredient(self):
        
        def split_active_inactive(sr_ingredient):
            inactive_start = pd.concat([sr_ingredient.str.find('Other'),
                                       sr_ingredient.str.find('Inactive'),
                                       sr_ingredient.str.find('Cosmetic Ingredients')],
                                       axis=1).max(axis=1)
            
            inactive_start = inactive_start.replace(-1, 0)
            inactive = [ingredient[start:] for (ingredient, start) in zip(sr_ingredient, inactive_start)]
            inactive = [ingredient[ingredient.find(':')+1:] for ingredient in inactive]                           
            active = [ingredient[:start] for (ingredient, start) in zip(sr_ingredient, inactive_start)]
            active = [ingredient[ingredient.rfind(':')+1:] for ingredient in active]
            return active, inactive
             
        def check_alphabetical(word_list, tol=1):
            if(len(word_list)) <= tol:
                return True
            count = 0
            for i in range(len(word_list) - 1):
                if word_list[i] > word_list[i + 1]:
                    count += 1
                if count > tol:
                    return False
            return True
        
        #split active and inactive ingredient
        self.df['ingredient'].fillna('', inplace=True)
        self.df['active_ingredient'], self.df['inactive_ingredient'] = split_active_inactive(self.df['ingredient'])
        #convert to list
        self.df['active_ingredient_list'] = self.df['active_ingredient'].apply(lambda x: [l.strip() for l in str(x).split(',') if l.lower().islower()])
        self.df['inactive_ingredient_list'] = self.df['inactive_ingredient'].apply(lambda x: [l.strip() for l in str(x).split(',') if l.lower().islower()])      
        #find number of ingredient
        self.df['n_inactive_ingredient'] = self.df['inactive_ingredient_list'].apply(lambda x: len(x))
        self.df['n_active_ingredient'] = self.df['active_ingredient_list'].apply(lambda x: len(x))
        #check if ingredients are listed alphabatically or perhaps by there quantity
        self.df['is_alphabatical'] = self.df['inactive_ingredient_list'].apply(check_alphabetical)

        
    def lookup_ingredients(self, lookup):
        
        print("processing all ingredients...")
        merged_ingredients = set(list(itertools.chain(*self.df['inactive_ingredient_list'].values)))
        merged_ingredients = merged_ingredients.union(
                             set(list(itertools.chain(*self.df['active_ingredient_list'].values))))
        lookup.find_matching_ingredient(merged_ingredients)
        ingredient_property = pd.DataFrame(index=merged_ingredients)
        print("find all ingredients information...")
        ingredient_property['matching'] = [lookup.lookup(ingredient, option='ingredient') for ingredient in merged_ingredients]
        ingredient_property['rating'] = [lookup.lookup(ingredient, option='rating') for ingredient in merged_ingredients]
        ingredient_property['category'] = [lookup.lookup(ingredient, option='category') for ingredient in merged_ingredients]
        
        # map origianal ingredient list to matched ingredient
        self.df['inactive_ingredient_matched_list'] = [[ingredient_property.loc[ingredient, 'matching'] 
                                                        for ingredient in ingredients]
                                                        for ingredients in self.df['inactive_ingredient_list'].values]
        self.df['active_ingredient_matched_list'] = [[ingredient_property.loc[ingredient, 'matching'] 
                                                        for ingredient in ingredients]
                                                        for ingredients in self.df['active_ingredient_list'].values]
        
        # map original ingredient list to ingredient rating
        self.df['inactive_ingredient_rating_list'] = [[ingredient_property.loc[ingredient, 'rating'] 
                                                        for ingredient in ingredients]
                                                        for ingredients in self.df['inactive_ingredient_list'].values]
        self.df['active_ingredient_rating_list'] = [[ingredient_property.loc[ingredient, 'rating'] 
                                                        for ingredient in ingredients]
                                                        for ingredients in self.df['active_ingredient_list'].values]
        
        # map original ingredient list to ingredient category
        self.df['inactive_ingredient_category_list'] = [[ingredient_property.loc[ingredient, 'category'] 
                                                        for ingredient in ingredients]
                                                        for ingredients in self.df['inactive_ingredient_list'].values]
        self.df['active_ingredient_category_list'] = [[ingredient_property.loc[ingredient, 'category'] 
                                                        for ingredient in ingredients]
                                                        for ingredients in self.df['active_ingredient_list'].values]
        
        def count_ingredient(col, prefix='count', mode='rating'):
            count_df = pd.DataFrame()
            if mode=='rating':
                count_df = pd.DataFrame.from_dict([dict(Counter(row))
                                                   for row in self.df[col].values])
            elif mode=='cat':
                # if an ingredient belongs to multiple category, we will increment all categories
                count_df = pd.DataFrame.from_dict([dict(Counter([cat for catlist in row for cat in catlist]))
                                                   for row in self.df[col].values])
            else:
                print('unknown mode in count_ingredient')
                return count_df
            
            count_df.set_index(self.df.index, inplace=True)
            count_df.fillna(0, inplace=True)
            count_df = count_df.add_prefix(prefix)
            return count_df
        
        # count inactive rating
        inactive_rating_count = count_ingredient('inactive_ingredient_rating_list', prefix='inactive_rating_count_')
        
        # count active rating
        active_rating_count = count_ingredient('active_ingredient_rating_list', prefix='active_rating_count_')
        
        # count inactive category
        inactive_category_count = count_ingredient('inactive_ingredient_category_list', 
                                                   prefix='inactive_cat_count_', mode='cat')
        
        # count active category
        active_category_count = count_ingredient('active_ingredient_category_list', 
                                                 prefix='active_cat_count_', mode='cat')
        
        # merge to main dateframe
        self.df = pd.concat([self.df, 
                             inactive_rating_count,
                               active_rating_count,
                             inactive_category_count,
                               active_category_count,], axis=1)
        
        # compute average ingredient rating
        def get_mean_rating(rating_df):
            mean_rating = np.zeros(rating_df.shape[0])
            for col in rating_df.columns.values:
                rating = float(col[-1])
                mean_rating += rating_df[col].values * rating
            mean_rating = mean_rating / rating_df.sum(axis=1).values
            return mean_rating
        
        self.df['inactive_mean_rating'] = get_mean_rating(inactive_rating_count)
        self.df['active_mean_rating'] = get_mean_rating(active_rating_count)
        
        del ingredient_property
        del inactive_rating_count, active_rating_count
        del inactive_category_count, active_category_count
        gc.collect()
        
    def basic_clean(self):        
        self.clean_size()
        self.clean_price()
        self.clean_ingredient()
        
    def get_df(self):
        return self.df

Read product data from disk

In [6]:
cols = ['product_names','product_category','brand','ingredient','size','price']
skin_care_df = pd.read_csv('../web_scraper/skin_care_products.csv', usecols = cols)
body_care_df = pd.read_csv('../web_scraper/body_care_products.csv', usecols = cols)
makeup_df = pd.read_csv('../web_scraper/makeup_products.csv', usecols = cols)

#### Before processing all products, let's sample a few products and check how the data cleaning is doing..

check basic data cleaning:

In [30]:
sample = skin_care_df.sample(10)
data_cleaner = product_df_cleaning(sample)
data_cleaner.basic_clean()
sample_cleaned = data_cleaner.get_df()
sample_cleaned

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,product_names,product_category,brand,ingredient,size,price,size_num,size_unit,avg_price,active_ingredient,inactive_ingredient,active_ingredient_list,inactive_ingredient_list,n_inactive_ingredient,n_active_ingredient,is_alphabatical
4419,Melissa Day Cream,Nighttime Moisturizer,Dr. Hauschka,"Water, Helianthus Annuus (Sunflower) Seed Oil...",1.00 fl. oz.,39.95,30.0,ml,1.331667,,"Water, Helianthus Annuus (Sunflower) Seed Oil...",[],"[Water, Helianthus Annuus (Sunflower) Seed Oil...",29,0,False
5574,"Sun Wrinkle Control Cream for Face SPF 50+, Ve...",Sunscreen,Clarins,"Active: Homosalate (10%), Oxybenzone (6%), O...",2.50 fl. oz.,33.00,74.0,ml,0.445946,"Homosalate (10%), Oxybenzone (6%), Octisala...","Water, C12-15 Alkyl Benzoate, Butyloctyl Sa...","[Homosalate (10%), Oxybenzone (6%), Octisalate...","[Water, C12-15 Alkyl Benzoate, Butyloctyl Sali...",40,5,False
2505,Age Arrest Eye Cream,Eye Cream & Treatment,Kate Somerville,"Water/Aqua/Eau, Isocetyl Stearate, Caprylic/...",0.50 fl. oz.,75.00,15.0,ml,5.000000,,"Water/Aqua/Eau, Isocetyl Stearate, Caprylic/...",[],"[Water/Aqua/Eau, Isocetyl Stearate, Caprylic/C...",64,0,False
3406,Age Perfect Anti-Sagging + Anti-Age Spot Hydra...,Daytime Moisturizer,L'Oreal Paris,"Active: Ensulizole (1.7%), Octinoxate (7.5%),...",2.50 fl. oz.,17.99,74.0,ml,0.243108,"Ensulizole (1.7%), Octinoxate (7.5%),","Water, Cyclohexasiloxane, Glycerin, Minera...","[Ensulizole (1.7%), Octinoxate (7.5%)]","[Water, Cyclohexasiloxane, Glycerin, Mineral O...",41,2,False
1075,Ultra Light Cleansing Oil,Cleansers,Neutrogena,"Mineral Oil, Isopropyl Isostearate, C12-C15 Al...",4.00 fl. oz.,9.99,118.0,ml,0.084661,,"Mineral Oil, Isopropyl Isostearate, C12-C15 Al...",[],"[Mineral Oil, Isopropyl Isostearate, C12-C15 A...",18,0,False
4445,Intense Moisturizing Cream,Nighttime Moisturizer,Mary Kay,"Water, Mineral Oil, Glycerin, Petrolatum, ...",1.80 fl. oz.,32.00,53.0,ml,0.603774,,"Water, Mineral Oil, Glycerin, Petrolatum, ...",[],"[Water, Mineral Oil, Glycerin, Petrolatum, Iso...",32,0,False
1247,Clean + Blackhead Eliminating Scrub for Oily S...,Exfoliants,Garnier Nutritioniste,Active Ingredients: 2% Salicylic Acid. Inactiv...,5.00 fl. oz.,5.99,148.0,ml,0.040473,2% Salicylic Acid.,"Water, Sodium Laureth Sulfate, Polyethylene...",[2% Salicylic Acid.],"[Water, Sodium Laureth Sulfate, Polyethylene, ...",18,1,False
3870,Multi-Perfecting Pore Corrector Gel Moisturizer,Nighttime Moisturizer,Algenist,"Water, Caprylic/Capric Triglyceride, Hydroxy...",2.00 fl. oz.,65.00,59.0,ml,1.101695,,"Water, Caprylic/Capric Triglyceride, Hydroxy...",[],"[Water, Caprylic/Capric Triglyceride, Hydroxye...",32,0,False
5026,Brilliant-C Brightening Serum,Serum,Merle Norman,"Cyclomethicone, Water, Disiloxane, Polyethy...",1.00 fl. oz.,48.00,30.0,ml,1.600000,,"Cyclomethicone, Water, Disiloxane, Polyethy...",[],"[Cyclomethicone, Water, Disiloxane, Polyethyle...",45,0,False
2107,Hawaiian Eye Gel Revitalizing Green Tea,Eyes,Alba Botanica,"Water, Aloe Barbadensis Leaf Juice, Glycerin...",1.00 fl. oz.,17.95,30.0,ml,0.598333,,"Water, Aloe Barbadensis Leaf Juice, Glycerin...",[],"[Water, Aloe Barbadensis Leaf Juice, Glycerin,...",19,0,False


Find all unique ingredients in these 10 products, look up these ingredients in dictionary, check the matching accuracy:

We can see for most ingredient, our matching algorithm find a reasonable match, however, there are a few mistakes:


In [8]:
merged_ingredients = set(list(itertools.chain(*sample_cleaned['inactive_ingredient_list'].values)))
ingredient_property = pd.DataFrame(index=merged_ingredients)
lookup.find_matching_ingredient(merged_ingredients)
ingredient_property['matching'] = [lookup.lookup(ingredient, option='ingredient') for ingredient in merged_ingredients]
ingredient_property['rating'] = [lookup.lookup(ingredient, option='rating') for ingredient in merged_ingredients]
ingredient_property['category'] = [lookup.lookup(ingredient, option='category') for ingredient in merged_ingredients]
ingredient_property

100%|██████████| 205/205 [00:19<00:00, 10.76it/s]


,matching,rating,category
Oenothera Biennis (Evening Primrose) Flower Extract,Anthemis nobilis flower extract,3,"[Skin-Soothing, Antioxidants]"
Ethoxydiglycol,ethoxydiglycol,2,[Miscellaneous]
Acrylates/C10-30 Alkyl Acrylate Crosspolymer,C10-30 alkyl acrylate crosspolymer,2,[Texture Enhancer]
Saccharomyces Cerevisiae Extract,Saccharomyces cerevisiae,1,[Skin-Replenishing]
Ethylene Brassylate,octyl salicylate,3,[Sunscreen Actives]
Polymnia Sonchifolia Root Juice,Mahonia aquifolium root extract,2,"[Antioxidants, Plant Extracts]"
LinumUsitatissimum (Linseed) Seed Extract,Linum usitatissimum extract,3,[Plant Extracts]
PEG-6 Caprylic/Capric Glycerides,PEG-60 almond glycerides,2,[Emollients]
Polysorbate 60,polysorbate 60,2,[Cleansing Agents]
Trimethylbenzenepropanol,methylpropanediol,2,"[Miscellaneous, Hydration]"


Look up ingredients for these sample product, and see the final cleaned dataframe.

In [31]:
data_cleaner.lookup_ingredients(lookup)
sample_cleaned = data_cleaner.get_df()
sample_cleaned

  0%|          | 0/251 [00:00<?, ?it/s]

processing all ingredients...


100%|██████████| 251/251 [00:22<00:00, 11.09it/s]

find all ingredients information...



/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:149: RuntimeWarning: invalid value encountered in true_divide


,product_names,product_category,brand,ingredient,size,price,size_num,size_unit,avg_price,active_ingredient,...,inactive_cat_count_Thickeners,inactive_cat_count_Thickeners/Emulsifiers,inactive_cat_count_Uncategorized,inactive_cat_count_Vitamins,active_cat_count_Anti-Acne,active_cat_count_Exfoliant,active_cat_count_Skin-Soothing,active_cat_count_Sunscreen Actives,inactive_mean_rating,active_mean_rating
4419,Melissa Day Cream,Nighttime Moisturizer,Dr. Hauschka,"Water, Helianthus Annuus (Sunflower) Seed Oil...",1.00 fl. oz.,39.95,30.0,ml,1.331667,,...,1.0,3,0.0,0.0,0.0,0.0,0.0,0.0,1.965517,NaN
5574,"Sun Wrinkle Control Cream for Face SPF 50+, Ve...",Sunscreen,Clarins,"Active: Homosalate (10%), Oxybenzone (6%), O...",2.50 fl. oz.,33.00,74.0,ml,0.445946,"Homosalate (10%), Oxybenzone (6%), Octisala...",...,1.0,1,2.0,2.0,0.0,0.0,0.0,5.0,1.725000,3.0
2505,Age Arrest Eye Cream,Eye Cream & Treatment,Kate Somerville,"Water/Aqua/Eau, Isocetyl Stearate, Caprylic/...",0.50 fl. oz.,75.00,15.0,ml,5.000000,,...,0.0,8,2.0,4.0,0.0,0.0,0.0,0.0,2.234375,NaN
3406,Age Perfect Anti-Sagging + Anti-Age Spot Hydra...,Daytime Moisturizer,L'Oreal Paris,"Active: Ensulizole (1.7%), Octinoxate (7.5%),...",2.50 fl. oz.,17.99,74.0,ml,0.243108,"Ensulizole (1.7%), Octinoxate (7.5%),",...,1.0,5,2.0,1.0,0.0,0.0,0.0,2.0,1.731707,2.5
1075,Ultra Light Cleansing Oil,Cleansers,Neutrogena,"Mineral Oil, Isopropyl Isostearate, C12-C15 Al...",4.00 fl. oz.,9.99,118.0,ml,0.084661,,...,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,2.000000,NaN
4445,Intense Moisturizing Cream,Nighttime Moisturizer,Mary Kay,"Water, Mineral Oil, Glycerin, Petrolatum, ...",1.80 fl. oz.,32.00,53.0,ml,0.603774,,...,0.0,5,0.0,1.0,0.0,0.0,0.0,0.0,2.125000,NaN
1247,Clean + Blackhead Eliminating Scrub for Oily S...,Exfoliants,Garnier Nutritioniste,Active Ingredients: 2% Salicylic Acid. Inactiv...,5.00 fl. oz.,5.99,148.0,ml,0.040473,2% Salicylic Acid.,...,0.0,1,2.0,0.0,1.0,1.0,1.0,0.0,2.111111,3.0
3870,Multi-Perfecting Pore Corrector Gel Moisturizer,Nighttime Moisturizer,Algenist,"Water, Caprylic/Capric Triglyceride, Hydroxy...",2.00 fl. oz.,65.00,59.0,ml,1.101695,,...,0.0,3,1.0,0.0,0.0,0.0,0.0,0.0,2.000000,NaN
5026,Brilliant-C Brightening Serum,Serum,Merle Norman,"Cyclomethicone, Water, Disiloxane, Polyethy...",1.00 fl. oz.,48.00,30.0,ml,1.600000,,...,1.0,6,3.0,1.0,0.0,0.0,0.0,0.0,2.155556,NaN
2107,Hawaiian Eye Gel Revitalizing Green Tea,Eyes,Alba Botanica,"Water, Aloe Barbadensis Leaf Juice, Glycerin...",1.00 fl. oz.,17.95,30.0,ml,0.598333,,...,0.0,1,0.0,3.0,0.0,0.0,0.0,0.0,2.263158,NaN


#### clean up skin care products and save to disk

In [ ]:
data_cleaner = product_df_cleaning(skin_care_df)
data_cleaner.drop_rows({'product_category': ['Cleansing Brushes & Devices']})
data_cleaner.basic_clean()
data_cleaner.lookup_ingredients(lookup)
skin_care_cleaned = data_cleaner.get_df()
skin_care_cleaned.to_csv('skin_care_cleaned.csv',index=False)

#### clean up body care products and save to disk

In [ ]:
data_cleaner = product_df_cleaning(body_care_df)
data_cleaner.basic_clean()
data_cleaner.lookup_ingredients(lookup)
body_care_cleaned = data_cleaner.get_df()
body_care_cleaned.to_csv('body_care_cleaned.csv',index=False)

#### clean up make products and save to disk

In [ ]:
data_cleaner = product_df_cleaning(makeup_df)
data_cleaner.drop_rows({'product_category': ['Makeup Brushes']})
data_cleaner.basic_clean()
data_cleaner.lookup_ingredients(lookup)
makeup_cleaned = data_cleaner.get_df()
makeup_cleaned.to_csv('makeup_cleaned.csv',index=False)